## Configuration

In [4]:
from collections import defaultdict
import numpy as np
import sys

In [5]:
USE_APPEND_MYMODULES = True

if USE_APPEND_MYMODULES is True:
    sys.path.append('./mypackage/')

from corpus import Corpus

In [6]:
max_l_length = 8
max_r_length = 5
min_count = 30
noun_score_threshold = 0.1

corpus_fname = "./data/2016-10-28_article_all_normed.txt"
corpus = Corpus(corpus_fname, iter_sent=True)
len(corpus)

176597

## L-R graph

L-R graph는 L 왼쪽에 어떤 R들이 등장하는지 확인하기 위한 그래프로 dict[L]에는 [R]들이 몇 번 오른쪽에 등장하였는지의 빈도수 입니다. 

max_l_length 기준으로 subword_l를 잘랐으며, subword_r의 길이가 max_r_length보다 길 경우에는 lr_graph에 저장하지 않습니다.

In [8]:
lr_graph = defaultdict(lambda: defaultdict(lambda: 0))

for sent in corpus:
    for token in sent.split():
        for e in range(1, min(max_l_length, len(token)) + 1):
            subword_l = token[:e]
            subword_r = token[e:]
            if len(subword_r) > max_r_length:
                continue
            lr_graph[subword_l][subword_r] += 1

In [9]:
_tmp = defaultdict(lambda: 0)
_tmp['a'] = 1
print(_tmp)

# defaultdict 는 카운팅 할 때 편하지만 

# get은 불편하므로
# _tmp.get("b", 0) 로 사용
#print(_tmp)

# 값을 가져올 때는 dict로 바꿔주자
ditc_tmp = dict(_tmp)

print(_tmp['b'])
print(_tmp)

defaultdict(<function <lambda> at 0x1236b8f28>, {'a': 1})
0
defaultdict(<function <lambda> at 0x1236b8f28>, {'b': 0, 'a': 1})


L인 word가 주어지면 R들의 분포를 가져오기 위해서 get 함수를 이용합시다.

In [12]:
def get_r_features(word):
    return lr_graph.get(word, {})

word = '박근혜'
for r, freq in get_r_features(word).items():
    print('%s - %s: %d' % (word, r, freq))
    break

박근혜 - : 4683


## R feature score table

강의자료에서 언급한, 세종말뭉치에 존재하는 r set들의 명사 가능 점수를 계산한 table을 로딩합니다. 

    str\tfloat
    
으로, 탭으로 구분되어 있습니다. 

In [14]:
with open('./data/noun_score_sejong', encoding='utf-8') as f:
    r_score = {}
    for line in f:
        r, score = line.strip().split('\t')
        score = float(score)
        r_score[r] = score
    print('num r features = %d' % len(r_score))

num r features = 2398


## Classify whether given word is noun or not

단어 word가 주어졌을 때, r features를 가져온 뒤, 이를 이용하여 명사 가능 점수를 계산해 봅시다.  

r_features를 가져온 뒤, 각 r의 명사 가능 점수를, 빈도수를 weight로 하여 가중평균 하였습니다.

In [15]:
def predict(word):
    r_features = get_r_features(word)
    
    score = 0
    norm = 0
    for r, freq in r_features.items():
        if (r in r_score) == False:
            continue
        score += r_score.get(r) * freq
        norm += freq
        
    if norm != 0:
        score /= norm
    
    return score


for word in ['박근', '박근혜', '박근혜대통령', '대통령', '정', '정부', '정부의', '알아', '알아냈']:
    print('noun score of %s = %.3f' % (word, predict(word)))

noun score of 박근 = 0.000
noun score of 박근혜 = 0.490
noun score of 박근혜대통령 = 0.821
noun score of 대통령 = 0.694
noun score of 정 = 0.346
noun score of 정부 = 0.628
noun score of 정부의 = 0.000
noun score of 알아 = -0.827
noun score of 알아냈 = -0.976
